Numba supports code generation for classes via the `numba.jitclass()` decorator. A class can be marked for optimization using this decorator along with a specification of the types of each field. We call the resulting class object a *jitclass*. All methods of a jitclass are compiled into nopython functions. **The data of a jitclass instance is allocated on the heap as a C-compatible structure so that any compiled functions can have direct access to the underlying data, bypassing the interpreter.**

## 1. Basic Usage

In [17]:
import numpy as np
from numba import jitclass, int32, float32

spec = [("value", int32), ("array", float32[:])]


@jitclass(spec)
class Bag(object):
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)
    
    @property
    def size(self):
        return self.array * value
    
    def increment(self, val):
        for i in range(self.size):
            self.array[i] = val
        return self.array

In [23]:
class Bag2(object):
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)
    
    @property
    def size(self):
        return self.array * value
    
    def increment(self, val):
        for i in range(self.size):
            self.array[i] = val
        return self.array

In [24]:
value = 10  # must be a global variable
mybag = Bag(value)
mybag2 = Bag2(value)

In [27]:
%timeit -r 10 -n 5 mybag.size

1.42 µs ± 849 ns per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [26]:
%timeit -r 10 -n 5 mybag2.size

1.73 µs ± 1.16 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


In the above example, a spec is provided as a list of 2-tuples. The tuples contain the name of the field and the Numba type of the field. Alternatively, user can use a dictionary (an OrderedDict preferably for stable field ordering), which maps field names to types.   
<br>
The definition of the class requires at least a `__init__` method for initializing each defined fields. Uninitialized fields contains garbage data. Methods and properties (getters and setters only) can be defined. They will be automatically compiled.

## 2. Support operations

The following operations of jitclasses work in both the interpreter and Numba compiled functions:

+ calling the jitclass class object to construct a new instance (e.g. `mybag = Bag(123)`);
+ read/write access to attributes and properties (e.g. `mybag.value`);
+ calling methods (e.g. `mybag.increment(3)`);

Using jitclasses in Numba compiled function is more efficient. Short methods can be inlined (at the discretion of LLVM inliner). Attributes access are simply reading from a C structure. Using jitclasses from the interpreter has the same overhead of calling any Numba compiled function from the interpreter. Arguments and return values must be unboxed or boxed between Python objects and native representation. Values encapsulated by a jitclass does not get boxed into Python object when the jitclass instance is handed to the interpreter. It is during attribute access to the field values that they are boxed.

## 3. Limitations

+ A jitclass class object is treated as a function (the constructor) inside a Numba compiled function.
isinstance() only works in the interpreter.
+ Manipulating jitclass instances in the interpreter is not optimized, yet.
+ Support for jitclasses are available on CPU only. (Note: Support for GPU devices is planned for a future release.)

## 4. Another nested example from stackoverflow

In [33]:
from numba import jitclass, int32, float64, deferred_type

@jitclass(spec=[("n", int32)])
class toto(object):
    def __init__(self, n):
        self.n = 42 + n
    
    def work(self, y):
        return y + self.n

    
toto_type = deferred_type()
toto_type.define(toto.class_type.instance_type)

@jitclass(spec=[("a", float64), ("b", float64), ("c", toto_type)])
class fifi(object):
    def __init__(self, combis):
        self.a = combis
        self.b = 2
        self.c = toto(combis)
    
    def sqrt(self, x):
        s = x
        for i in range(self.a):
            s = (s + x / s) / 2.0
        return s

In [34]:
class toto2(object):
    def __init__(self, n):
        self.n = 42 + n
    
    def work(self, y):
        return y + self.n

class fifi2(object):
    def __init__(self, combis):
        self.a = combis
        self.b = 2
        self.c = toto(combis)
    
    def sqrt(self, x):
        s = x
        for i in range(self.a):
            s = (s + x / s) / 2.0
        return s

In [35]:
obj = fifi(200)
obj2 = fifi2(200)

In [37]:
%timeit -r 10 -n 5 obj.sqrt(5)

3.94 µs ± 863 ns per loop (mean ± std. dev. of 10 runs, 5 loops each)


In [38]:
%timeit -r 10 -n 5 obj2.sqrt(5)

21.3 µs ± 337 ns per loop (mean ± std. dev. of 10 runs, 5 loops each)
